In [56]:
import httplib2
import oauth2
import urllib3
import types
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gender_detector import GenderDetector 
import psycopg2, psycopg2.extras
from causalinference import CausalModel
from causalinference.utils import random_data
import httplib
import base64
import json # For Microsoft Face API
import urllib as urllib # For Microsoft Face API
import time 
import csv
import datetime 
import random
import os.path
from sklearn.metrics import cohen_kappa_score

file1 = "./csv_files/2_6_2018_UpworkGroundTruth_1.csv"
file2 = "./csv_files/2_6_2018_UpworkGroundTruth_2.csv" # This contains data on education level as well 

df_1 = pd.read_csv(file1)
df_2 = pd.read_csv(file2)

df_1 = df_1[['user_count', 'gender', 'age_range']]
df_2 = df_2[['user_count', 'gender', 'age_range', 'education']]


gender_truth_1 = []
gender_truth_2 = []
age_truth_1 = []
age_truth_2 = []
education_truth = []

for index, row in df_1.iterrows():
    gender_truth_1.append(row['gender'])
    age_truth_1.append(row['age_range'])

for index, row in df_2.iterrows():
    gender_truth_2.append(row['gender'])
    age_truth_2.append(row['age_range'])
    education_truth.append(row['education'])

kappa_gender = cohen_kappa_score(gender_truth_1, gender_truth_2)
kappa_age = cohen_kappa_score(age_truth_1, age_truth_2)

print(kappa_gender)
print(kappa_age)


0.985372539029
0.677283148753


In [95]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score

# Compile dataset of estimated values of gender and age
gender_estimated = []
age_id_estimated = []
gender_truth = []
age_id_truth = []

ground_truth_file = './csv_files/2_6_2018_UpworkGroundTruth_1.csv'
estimated_file = './csv_files/estimated_final_gender_groundtruth1_2017_12_12_upwork_analysis_unitedstates_allskills.csv'
df_ground_truth = pd.read_csv(ground_truth_file)
df_estimated = pd.read_csv(estimated_file)

for index, row in df_estimated.iterrows():
    gender_estimated.append(row['est_gender'])
    age_id_estimated.append(str(row['est_age']))
    
for index, row in df_ground_truth.iterrows():
    gender_truth.append(row['gender'])
    age = row['age_range']
    # Reformat age ranges
    
    if age == "0-15":
        age = 0
    elif age == "16-19":
        age = 1
    elif age == "20-24":
        age = 2
    elif age == "25-34":
        age = 3
    elif age == "35-44":
        age = 4
    elif age == "45-54":
        age = 5
    elif age == "55-64":
        age = 6
    elif age == "65 and older":
        age = 7
        
    age_id_truth.append(str(age))

print age_id_estimated
print age_id_truth

gender_truth = np.array(gender_truth[:200]) # Limit to rows that have data
age_id_truth = np.array(age_id_truth[:200])
gender_estimated = np.array(gender_estimated[:200])
age_id_estimated = np.array(age_id_estimated)

gender_precision = precision_recall_fscore_support(gender_truth, gender_estimated, average='weighted')
age_precision = precision_recall_fscore_support(age_id_truth, age_id_estimated, average='weighted')

print "Gender precision: {0}".format(gender_precision)
print "Age precision: {0}".format(age_precision)

['3', '4', '3', '3', '5', '5', '5', '2', '7', 'error', 'error', 'error', '3', 'error', 'error', '3', '5', '3', '3', '3', 'error', '4', '3', '3', '3', '4', '5', 'error', '4', '3', '3', '2', '3', 'error', 'error', '3', '7', '4', '4', '3', '3', '3', '4', '4', '3', '4', '2', '2', '6', '2', '3', 'error', '3', '4', '3', '4', '2', '4', '4', '2', '4', '4', '4', '6', '3', '3', '4', '6', '2', '3', '5', '3', '3', '3', '4', '3', '5', '3', '1', '4', '1', '3', '4', '3', '2', '4', '3', 'error', '5', '4', '3', '5', '3', '3', '4', '5', '7', '2', '3', '3', '3', '3', 'error', '3', '2', '3', '3', '3', '3', '3', '3', '4', '3', '3', '4', '3', '4', '4', '3', '5', '3', '5', '3', '4', '3', '3', '5', '4', '4', '3', '4', '2', '2', '3', '6', '4', '2', '2', 'error', '2', '3', 'error', '3', '3', '3', '3', '4', 'error', '4', '2', '4', '4', '3', '3', '0', '1', '3', '3', '4', '3', '2', '4', '4', '4', '4', '3', '4', '7', 'error', '6', '4', '7', '3', '3', 'error', '3', '3', '6', '3', 'error', '2', '5', '7', '3', '2', 'e

In [30]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score

# Compile dataset of estimated values of gender and age
gender_estimated = []
gender_final_estimated = []
age_id_estimated = []
age_estimated = []
education_estimated = []

analysis_file = './csv_files/formatted_analysis_df_2017_12_12_upwork_analysis_unitedstates_allskills.csv'
df_analysis = pd.read_csv(analysis_file)

# Get list of user counts in ground truth dataset
user_counts = []

for index, row in df_1.iterrows():
    user_counts.append(row['user_count'])

for index, row in df_analysis.iterrows():
    if row['user_count'] in user_counts:
        gender_estimated.append(row['final_gender'])
        age_id_estimated.append(row['new_age_range_id'])
        education_estimated.append(row['education'])

for item in age_id_estimated:
    if item == '0':
        age_estimated.append('0-15')
    elif item == '1':
        age_estimated.append('16-19')
    elif item == '2':
        age_estimated.append('20-24')
    elif item == '3':
        age_estimated.append('25-34')
    elif item == '4':
        age_estimated.append('35-44')
    elif item == '5':
        age_estimated.append('45-54')
    elif item == '6':
        age_estimated.append('55-64')
    elif item == '7':
        age_estimated.append('65 and older')
    else:
        age_estimated.append('error')

for item in gender_estimated:
    if item == "unknown":
        gender_final_estimated.append("error")
    elif item == "mostly_male":
        gender_final_estimated.append("male")
    elif item == "mostly_female":
        gender_final_estimated.append("female")
    elif item == "andy":
        gender_final_estimated.append("error")
    else:
        gender_final_estimated.append(item)

print "These are the estimated final genders"
print gender_final_estimated 

gender_true = np.array(gender_truth_1[:200]) # Limit to rows that have data
age_true = np.array(age_truth_1[:200])
gender_final_estimated = np.array(gender_final_estimated)
age_estimated = np.array(age_estimated)
# education_true = np.array(education_truth[:200])

gender_precision = precision_recall_fscore_support(gender_true, gender_final_estimated, average='macro')
age_precision = precision_recall_fscore_support(age_true, age_estimated, average='macro')
# education_precision = precision_recall_fscore_support(education_true, education_estimated, average='macro')

print "Gender precision: {0}".format(gender_precision)

print "Age precision: {0}".format(age_precision)

# print "Education precision: {0}".format(education_precision)

# Check the instances where the ground truth set does not match the estimated gender

i = 0

for result in gender_final_estimated:
    if gender_truth_1[i] != result:
        print user_counts[i]
    i += 1

These are the estimated final genders
['female', 'female', 'male', 'male', 'male', 'male', 'male', 'male', 'female', 'female', 'female', 'male', 'female', 'female', 'male', 'male', 'male', 'male', 'male', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'female', 'female', 'female', 'male', 'male', 'female', 'male', 'female', 'male', 'male', 'male', 'female', 'male', 'male', 'male', 'male', 'female', 'male', 'female', 'male', 'female', 'female', 'male', 'male', 'female', 'male', 'female', 'female', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'male', 'male', 'male', 'female', 'male', 'female', 'female', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'female', 'female', 'male', 'male', 'male', 'male', 'female', 'male', 'female', 'male', 'error', 'male', 'male', 'female', 'female', 'male', 'male', 'male', 'female', 'male', 'female', 'male', 'male', 'male', 'male', 'female', 'female', 'female', 'male', 'female', 'male', 

ValueError: Found input variables with inconsistent numbers of samples: [200, 201]